In [ ]:
import tensorflow as tf
import os
from bing_image_downloader import downloader
import cv2
import imghdr
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.regularizers import l2

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image_dataset_from_directory

import random
np.set_printoptions(suppress=True)

In [ ]:
# downloader.download("Hagia Sophia — Istanbul, Turkey", limit=250, output_dir="images")
# downloader.download("Dancing House — Prague, Czech Republic", limit=250, output_dir="images")
# downloader.download("Château de Chenonceau — Chenonceaux, France", limit=250, output_dir="images")
# downloader.download("The Colosseum — Rome, Italy", limit=250, output_dir="images")
# downloader.download("St. Basil’s Cathedral — Moscow, Russia", limit=250, output_dir="images")

In [ ]:
data_dir = "images"
image_exts = ["jpeg", "jpg", "bmp", "png"]

In [ ]:
# os.listdir(data_dir)

In [ ]:
# os.listdir(os.path.join(data_dir,"Chateau de Chenonceau"))

In [ ]:
# img = cv2.imread(os.path.join(data_dir, "Chateau de Chenonceau", "Image_10.jpg"))
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.show()
# img.shape

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(
    "images",
    labels="inferred",
    label_mode="categorical",
    image_size=(256, 256),
    batch_size=32,   
)
data

In [ ]:
data_iterator = data.as_numpy_iterator()
data_iterator

In [ ]:
batch = data_iterator.next()
len(batch)
batch[0].shape, batch[1].shape

In [ ]:
# fig, ax = plt.subplots(ncols=10, figsize=(20,20))
# for idx, img in enumerate(batch[0][:10]):
#   ax[idx].imshow(img.astype(int))
#   ax[idx].title.set_text(batch[1][idx])

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "images",
    labels="inferred",
    validation_split=0.3,
    subset="training",
    seed=123,
    label_mode="categorical",
    image_size=(256, 256),
    batch_size=32,
)
val_test_ds = tf.keras.utils.image_dataset_from_directory(
    "images",
    labels="inferred",
    validation_split=0.3,
    subset="validation",
    seed=123,
    label_mode="categorical",
    image_size=(256, 256),
    batch_size=32,   
)
tf.data.experimental.cardinality(val_test_ds).numpy()

In [ ]:
total_samples = val_test_ds.cardinality().numpy()
val_num = int(0.7*total_samples)
test_num = int(0.3*total_samples)
val_num, test_num

In [ ]:
val_ds = val_test_ds.take(val_num)
test_ds = val_test_ds.skip(val_num).take(test_num)
val_ds, test_ds

In [ ]:
main_ds = tf.keras.utils.image_dataset_from_directory(
    "images",
    labels="inferred",
    seed=123,
    label_mode="categorical",
    image_size=(256, 256),
    batch_size=32,
)

images_count_val = [0,0,0,0,0]
images_count_test = [0,0,0,0,0]

val__images = []
val__labels = []

test_images = []
test_labels = []

train_images = []
train_labels = []

for image,label in main_ds.unbatch():

    if images_count_test[np.argmax(label.numpy())] < 85:
        train_images.append(image)
        train_labels.append(label)
        images_count_test[np.argmax(label.numpy())] = images_count_test[np.argmax(label.numpy())] + 1

    elif images_count_val[np.argmax(label.numpy())] < 15:
        val__images.append(image)
        val__labels.append(label)
        images_count_val[np.argmax(label.numpy())] = images_count_val[np.argmax(label.numpy())] + 1
    
    else:
        test_images.append(image)
        test_labels.append(label)

train_ds_balanced = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds_balanced = train_ds_balanced.shuffle(100)
train_ds_balanced = train_ds_balanced.batch(32)

val_ds_balanced = tf.data.Dataset.from_tensor_slices((val__images, val__labels))
val_ds_balanced = val_ds_balanced.shuffle(100)
val_ds_balanced = val_ds_balanced.batch(32)

test_ds_balanced = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_ds_balanced = test_ds_balanced.shuffle(100)
test_ds_balanced = test_ds_balanced.batch(32)

train_ds_balanced

In [ ]:
import tensorflow as tf

class_counts = {}

for _, labels in val_ds_balanced:
    for label in labels.numpy():
        class_idx = label.argmax()  
        if class_idx not in class_counts:
            class_counts[class_idx] = 1
        else:
            class_counts[class_idx] += 1

for class_idx, count in class_counts.items():
    print(f"Class {class_idx}: {count} samples")


In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),  # RandomFlip layer
        tf.keras.layers.RandomTranslation(
            height_factor=(-0.2, 0.3), width_factor=(-0.2, 0.3)
        ),  # RandomTranslation layer
        tf.keras.layers.RandomRotation(factor=0.2),  # RandomRotation layer
        # tf.keras.layers.RandomZoom(height_factor=(0.2, 0.3)),  # RandomZoom layer
        tf.keras.layers.RandomContrast(factor=(0.2)),  # RandomContrast layer
        tf.keras.layers.RandomBrightness(factor=(0.2)),  # RandomBrightness layer
    ]
)
AUTOTUNE = tf.data.AUTOTUNE
train_ds_augmented = train_ds_balanced.map(
    lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE
)
train_ds_augmented2 = train_ds_balanced.map(
    lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE
)
train_ds_augmented3 = train_ds_balanced.map(
    lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE
)
train_ds_augmented4 = train_ds_balanced.map(
    lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE
)

In [ ]:
train_ds_merged = train_ds_balanced.concatenate(train_ds_augmented)
train_ds_merged = train_ds_merged.concatenate(train_ds_augmented2)
# train_ds_merged = train_ds_merged.concatenate(train_ds_augmented3)
# train_ds_merged = train_ds_merged.concatenate(train_ds_augmented4)

train_ds_merged

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

classes = [0, 1, 2, 3, 4]
y = np.array([label.numpy() for _, label in train_ds_merged.unbatch()])
y = np.argmax(y, axis=1)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
class_weights

In [ ]:
class_weights_dict = {}
for i, weight in enumerate(class_weights):
    class_weights_dict[i] = weight
class_weights_dict

In [ ]:
class_names = [0,1,2,3,4]
plt.figure(figsize=(10, 10))
for images, labels in train_ds_merged.take(4):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        label_index = np.argmax(labels[i].numpy()) #
        plt.title(class_names[int(label_index)])  
        plt.axis("off")

In [ ]:
train_ds_merged = train_ds_merged.map(lambda x, y: (x/255, y))
val_ds_balanced = val_ds_balanced.map(lambda x, y: (x/255, y))
test_ds_balanced = test_ds_balanced.map(lambda x, y: (x/255, y))

sample = val_ds_balanced.as_numpy_iterator().next()
sample[0].max()

In [ ]:
model = Sequential()

model.add(Conv2D(16, 7, activation="relu", padding="same", input_shape=(256, 256, 3)))
model.add(MaxPooling2D(2))
model.add(Dropout(0.25))  

model.add(Conv2D(32, 3, activation="relu", padding="same"))
model.add(MaxPooling2D())
model.add(Dropout(0.25))  

model.add(Conv2D(16, 3, activation="relu", padding="same"))
model.add(MaxPooling2D())
model.add(Dropout(0.25))  

model.add(Flatten())

model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))  

model.add(Dense(units=5, activation="softmax"))

In [ ]:
model.compile("adam", loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# model.summary()

In [ ]:
# from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
# callbacks = [
#         EarlyStopping(patience=10, monitor="val_loss", verbose=1),
#         TensorBoard(log_dir="logs", histogram_freq=1),
#     ]
# hist = model.fit(train_ds_merged, epochs=100, validation_data=val_ds_balanced, class_weight=class_weights_dict, callbacks=callbacks)
model.load_weights('./checkpoints/my_checkpoint')

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

true_labels = []
predicted_labels = []

for batch in test_ds_balanced:
    X, y = batch
    yhat = model.predict(X)
    true_labels.extend(np.argmax(y, axis=1))
    predicted_labels.extend(np.argmax(yhat, axis=1))

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
F1 = f1_score(true_labels, predicted_labels, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1: ", F1)

In [ ]:
import seaborn as sns

conf_matrix = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=[
        "Chateau de Chenonceau",
        "Dancing House",
        "Hagia Sophia",
        "St Basils Cathedral",
        "The Colosseum",
    ],
    yticklabels=[
        "Chateau de Chenonceau",
        "Dancing House",
        "Hagia Sophia",
        "St Basils Cathedral",
        "The Colosseum",
    ],
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# img = cv2.imread("st5.jpg")
# image_resized = tf.image.resize(img,(256,256))

# print(np.expand_dims(image_resized/255,0).shape)
# yhat = model.predict(np.expand_dims(image_resized/255,0))
# print(yhat)

# class_index = yhat.argmax(axis=1)
# if class_index == 0:
#     print("Chateau de Chenonceau")
# elif class_index == 1:
#     print("Dancing House")
# elif class_index == 2:
#     print("Hagia Sophia")
# elif class_index == 3:
#     print("St Basils Cathedral")
# elif class_index == 4:
#     print("The Colosseum")

# plt.imshow(image_resized.numpy().astype(int))
# plt.show()


In [ ]:
# test ds ne sme biti normalizovan!
class_names = ["Chateau", "Dancing", "Hagia", "St Basils", "Colosseum"]
plt.figure(figsize=(10, 10))
for images, labels in test_ds_balanced.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        label_index = np.argmax(model.predict(np.expand_dims(images[i],0))) #
        plt.title(f"PR: {class_names[int(label_index)]} -> ST: {class_names[np.argmax(labels[i].numpy())]}")  
        plt.axis("off")

In [ ]:
# TSNE implementacija
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import manifold
import tensorflow as tf

In [ ]:
# load dataset
# data = tf.keras.utils.image_dataset_from_directory(
#     "images",
#     labels="inferred",
#     label_mode="categorical",
#     image_size=(256, 256),
#     batch_size=32,   
# )

# Initialize lists to store features and targets
features = []
targets = []

# Iterate through the dataset to collect features and targets
for batch_features, batch_targets in data:
    features.extend(batch_features.numpy())
    targets.extend(batch_targets.numpy())

# Convert the lists to numpy arrays
features = np.array(features)
targets = np.array(targets)
targets = targets.astype(int)




In [ ]:
features = features.reshape(features.shape[0],-1)
features.shape

In [ ]:
# dimensionality reduction using t-SNE
tsne = manifold.TSNE(
    n_components=2,
)
# fit and transform
mnist_tr = tsne.fit_transform(features)
# transformed_data is a 2D numpy array of shape (30000, 2)

In [ ]:
targets = np.argmax(targets, axis=1)

In [ ]:
mnist_tr.shape
targets.shape

In [ ]:
# create dataframe
cps_df = pd.DataFrame(columns=['CP1', 'CP2', 'target'],
                       data=np.column_stack((mnist_tr, 
                                            targets)))
# cast targets column to int
cps_df.loc[:, 'target'] = cps_df.target.astype(int)
cps_df.head()

In [ ]:
buildings_map = {
    0: "Chateau",
    1: "Dancing",
    2: "Hagia",
    3: "St Basils",
    4: "The Colosseum",
}
# map targets to actual clothes for plotting
cps_df.loc[:, "target"] = cps_df.target.map(buildings_map)

In [ ]:
cps_df.target.value_counts().plot(kind='bar')

In [ ]:
grid = sns.FacetGrid(cps_df, hue="target", height=6)
grid.map(plt.scatter, 'CP1', 'CP2').add_legend()